In [ ]:
pip install azure-ai-documentintelligence

In [ ]:
import io, os, json
from azure.ai.documentintelligence import DocumentIntelligenceClient
from azure.core.credentials import AzureKeyCredential
from azure.core.serialization import AzureJSONEncoder

In [ ]:
# ── 1. Client + paths ────────────────────────────────────────────
key = "1waRdyOKyqc6V4DJqGPoDY6Z1TVAKzGWMKvd52IUvFvcB9H0Y5fhJQQJ99BEACYeBjFXJ3w3AAALACOGVxC5"
endpoint      = "https://usecase3docintel.cognitiveservices.azure.com/"
 
client = DocumentIntelligenceClient(endpoint, AzureKeyCredential(key))
 
input_folder  = "abfss://bronzedoc@conusecase3.dfs.core.windows.net/"
output_folder = "abfss://json@conusecase3.dfs.core.windows.net/
"
 
# ── 2. Read DOC/DOCX via Spark binaryFile ───────────────────────
doc_files = (
    spark.read.format("binaryFile")
    .option("pathGlobFilter", "*.docx")
    .load(input_folder)
)

# ── 3. Process each DOCX ─────────────────────────────────────────
for row in doc_files.collect():
    doc_path  = row.path
    doc_bytes = row.content
    print(f"\nProcessing: {doc_path}")

    # Analyse with prebuilt‑read (good for .docx, .doc)
    poller = client.begin_analyze_document(
        model_id="prebuilt-read",
        body=io.BytesIO(doc_bytes)
    )
    result = poller.result()

    # ── Build the output dict ────────────────────────────────────
    out = {
        "file": os.path.basename(doc_path),
        "styles": [],
        "pages": []
    }

    # A) Create every page dict (lines only)
    for page in result.pages:
        page_dict = {
            "page_number": page.page_number,
            "lines": [line.content for line in page.lines],
            "tables": []  # Optional: you can omit if not using tables
        }
        out["pages"].append(page_dict)

    # ── 4. Serialise & save to ADLS ──────────────────────────────
    json_str = json.dumps(out, indent=4, cls=AzureJSONEncoder)
    json_name = os.path.splitext(os.path.basename(doc_path))[0] + ".json"
    output_path = os.path.join(output_folder, json_name)

    mssparkutils.fs.put(output_path, json_str, overwrite=True)
    print(f"✓ Saved extracted JSON → {output_path}")

print("\nAll DOCX files processed and JSONs saved.")
